### Setup

In [1]:
import os, sys
HOME = os.environ['HOME']  # change if necessary
sys.path.append(f'{HOME}/Finite-groups/src')

In [2]:
import torch as t
import numpy as np
from matplotlib import pyplot as plt
import json
from itertools import product
from jaxtyping import Float
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import plotly.graph_objects as go
import copy
import math
from itertools import product
import pandas as pd
from typing import Union
from einops import repeat
from huggingface_hub import snapshot_download
from huggingface_hub.utils import disable_progress_bars


from model import MLP3, MLP4, InstancedModule
from utils import *
from group_data import *
from model_utils import *
from group_utils import *
from irrep_bounds import *
from bound_utils import *
%load_ext autoreload
%autoreload 2

Syntax warning: Unbound global variable in /usr/share/gap/pkg/browse/PackageIn\
fo.g:73
  if not IsKernelExtensionAvailable("Browse", "ncurses") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
Syntax warning: Unbound global variable in /usr/share/gap/pkg/edim/PackageInfo\
.g:60
  if not IsKernelExtensionAvailable("EDIM","ediv") then
         ^^^^^^^^^^^^^^^^^^^^^^^^^^


In [3]:
t.manual_seed(42)    # clustering depends on random seed
np.random.seed(42)

In [4]:
t.set_grad_enabled(False)

In [5]:

device = t.device("cuda" if t.cuda.is_available() else "cpu")
# MODEL_DIR = '2024-08-14_21-24-30_gapS_5_'
MODEL_DIR = '2024-09-18_23-59-00_gapS5_MLP2_128_wd2e-4_ubias'
# MODEL_DIR = '2024-09-22_22-12-21_gapF11_MLP2_256_wd1e-4_ubias'
# MODEL_DIR = '2024-09-23_01-06-18_A5x2_MLP2_128_wd1e-4_ubias'
# MODEL_DIR = '2024-09-23_02-50-16_sg96_227_MLP2_128_wd1e-4_ubias'

disable_progress_bars()
local_dir = f'{HOME}/models/{MODEL_DIR}'
if not os.path.exists(local_dir):
    snapshot_download(repo_id=f'wiwu2390/{MODEL_DIR}', local_dir=local_dir)
models, params = load_models(local_dir,) #sel='final')
models = models[-1]  # get last checkpoint
data = GroupData(params)
group = data.groups[0]

/home/wilson/Finite-groups/src/model_utils.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(t.load(model_path, map_location=device))


Intersection size: 14400/14400 (1.00)
Added 14400 elements from intersection
Added 0 elements from group 0: gapS(5)
Taking random subset: 5760/14400 (0.40)
Train set size: 5760/14400 (0.40)


In [6]:
save = t.load('../data/acc_bound_S5_new.pt')
locals().update(save)

/tmp/ipykernel_12705/3089656520.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  save = t.load('../data/acc_bound_S5_new.pt')


In [45]:
irrep_idxs = []
for instance in tqdm(range(len(models))):
    model = models[instance]
    irreps, irrep_idx_dict = get_neuron_irreps(model, group, norm_thresh=1)
    irrep_idxs.append(irrep_idx_dict)

100%|██████████| 100/100 [00:24<00:00,  4.16it/s]


In [41]:
ideal_models = MLP2.stack(ideal_models)

In [65]:
def model_subidx(model, idxs):
    if not isinstance(model, MLP4):
        model = model.fold_linear()
    ln, rn, un = model.get_neurons()
    ln, rn, un = ln[:,:,idxs], rn[:,:,idxs], un[:,:,idxs]
    ret = MLP4(model.params)
    ret.embedding_left = nn.Parameter(ln.clone())
    ret.embedding_right = nn.Parameter(rn.clone())
    ret.unembedding = nn.Parameter(un.clone().mT)
    ret.unembed_bias = nn.Parameter(model.unembed_bias.clone())
    return ret

In [ ]:
def equivariance(models, group):
    N = len(group)
    models = models.to(device)
    test_inputs = t.tensor(list(product(range(N), repeat=2)), device=device)
    output = models(test_inputs)
    vars = []
    for i in range(N):
        logit_vals = []
        for j, k in product(range(N), repeat=2):
            if group.mult_idx(j, k) == i:
                # output is (N^2, instance, N)
                logit_vals.append(output[N * j + k, :, i])
        logit_vals = t.stack(logit_vals, dim=1)  # (instance, N)
        vars.append(logit_vals.var(dim=1))   # (instance)
    vars = t.stack(vars, dim=1)  # (instance, N)
    return vars.mean(dim=1)

In [35]:
orig_equi = equivariance(models, group)

In [42]:
ideal_equi = equivariance(ideal_models, group)

In [43]:
ideal_equi

tensor([3.2630e-11, 3.2069e-04, 3.2964e-11, 5.5808e-12, 8.9564e-15, 1.7913e-15,
        6.2769e-12, 7.5545e-12, 3.3498e-11, 2.2395e-12, 6.6327e-12, 4.1209e-12,
        4.1422e-12, 2.7172e-04, 5.8153e-12, 0.0000e+00, 5.6104e-12, 4.9695e-04,
        7.3804e-12, 0.0000e+00, 6.3237e-12, 2.8613e-12, 8.3456e-03, 1.7913e-15,
        3.2617e-11, 0.0000e+00, 4.2399e-12, 1.1711e-11, 9.8181e-05, 0.0000e+00,
        4.7856e-12, 0.0000e+00, 3.4942e-11, 0.0000e+00, 4.1352e-12, 1.9201e-04,
        6.1087e-04, 1.0777e-11, 0.0000e+00, 1.0515e-04, 5.6488e-12, 3.9476e-12,
        3.1355e-11, 6.0247e-12, 1.7913e-15, 7.1651e-15, 6.8205e-12, 7.1250e-05,
        5.8940e-12, 7.1651e-15, 7.1651e-15, 4.0937e-12, 2.9968e-11, 5.7868e-12,
        2.3483e-04, 0.0000e+00, 1.7913e-15, 3.6337e-12, 3.9872e-12, 4.3435e-12,
        5.3966e-12, 8.1314e-12, 2.1066e-04, 1.9543e-04, 6.0131e-04, 0.0000e+00,
        4.7892e-12, 3.3201e-11, 7.6814e-04, 7.1651e-15, 1.7674e-11, 1.4640e-12,
        1.7446e-12, 0.0000e+00, 3.3450e-

In [48]:
ideal_model = ideal_models[1]

In [76]:
submodel = model_subidx(ideal_model, irrep_idxs[1]['4d-1'])
submodel.unembed_bias = nn.Parameter(t.zeros_like(submodel.unembed_bias))

In [ ]:
equivariance(ideal_model, group)

tensor([0.0003])

In [82]:
equivariance(submodel, group)

tensor([[12.0680, 12.0680, 12.0296, 12.0296, 12.0556, 12.0556, 12.0432, 12.0432,
         12.0296, 12.0296, 12.0556, 12.0556, 12.0432, 12.0432, 12.0680, 12.0680,
         12.0556, 12.0556, 12.0432, 12.0432, 12.0680, 12.0680, 12.0296, 12.0296,
         12.0680, 12.0680, 12.0545, 12.0545, 12.0680, 12.0680, 12.0680, 12.0680,
         12.0545, 12.0545, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680,
         12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0545, 12.0545,
         12.0296, 12.0296, 12.0160, 12.0160, 12.0420, 12.0420, 12.0545, 12.0545,
         12.0160, 12.0160, 12.0420, 12.0420, 12.0545, 12.0545, 12.0296, 12.0296,
         12.0420, 12.0420, 12.0545, 12.0545, 12.0296, 12.0296, 12.0160, 12.0160,
         12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680,
         12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680,
         12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680, 12.0680,
         12.0556, 12.0556, 1

tensor([0.0003])

In [83]:
model = models[1]
irreps, irrep_idx_dict = get_neuron_irreps(model, group, norm_thresh=0.1)
vecs, avar = get_neuron_vecs(model, group, irreps, irrep_idx_dict, strict=False, num_clusters=2, verbose=True, stab_thresh=0.3)
# irrep_acc, irrep_time, all_zeroed = irrep_acc_bound(model, group, irreps, irrep_idx_dict, vecs)
unif_vecs, zeroed_irreps = get_unif_vecs(group, irreps, vecs, verbose=True, stab_thresh=0.3)
ideal, _ = get_idealized_model(model, irreps, irrep_idx_dict, unif_vecs, verbose=True)

1d-0
1-r2 90th percentile 0.0
a variance: 0.0
b variance: 1.0
c variance: 1.0
d variance: 0.0
a vs d tensor(0.)
b_hat diff tensor(0.)
c_hat diff tensor(0.)
b_mean tensor([[1.0000],
        [2.0984]])
c_mean tensor([[-1.],
        [ 0.]])

4d-0
5d-0
6d-0
5d-1
4d-1
1-r2 90th percentile 0.9857432246208191
a variance: 1.4956707161672966e-07
b variance: 0.9223583936691284
c variance: 0.9651285409927368
d variance: 1.6265930469216983e-07
a vs d tensor(6.4716e-07)
b_hat diff tensor(0.0423)
c_hat diff tensor(0.0423)
b_mean tensor([[-0.5381,  0.3449,  0.4183, -0.5977],
        [-0.6864,  0.1620, -0.1620,  0.6864]])
c_mean tensor([[ 0.5183, -0.4740, -0.3761,  0.5123],
        [ 0.6864, -0.1620,  0.1620, -0.6864]])

1d-1
UNIF VECS 1d-0
UNIF VECS 4d-1
4d-1 zeroed 24 47 stab 24
4d-1 zeroed 0 47 stab 24
coef tensor([6.8272e+00, 1.1206e-05, 6.8330e+00, 6.8457e+00, 6.8547e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.8171e+00, 6.7989e+00,
        6.8147e+00, 0.0000e+00, 0.

/home/wilson/Finite-groups/src/irrep_bounds.py:395: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3697.)
  un_hat.T @ un_hat,


In [ ]:
submodel(t.tensor([[0, 0]]))

tensor([[[12.0680,  5.9817, -0.1083, -0.1083, -0.1083,  5.9780,  3.0700,
          -3.0163, -0.1083, -3.0163, -0.1083,  3.0700, -0.1083, -0.1083,
           8.8897,  5.9817,  2.7997, -0.1083, -3.0163,  3.0700,  5.9817,
           9.1600, -0.1083, -3.0163,  6.2507,  0.1643, -5.9257, -5.9257,
          -5.9257,  0.1606,  3.0700, -3.0163, -5.9257, -5.9257, -5.9257,
           0.1606, -0.1083, -0.1083,  3.0724,  3.0724, -3.0177, -3.0177,
          -3.0163,  3.0700,  0.1643,  6.2507, -5.9257, -5.9257,  3.0700,
          -3.0163, -0.1083, -3.0163, -0.1083,  3.0700,  8.8874,  2.8011,
          -0.1083, -0.1083, -0.1083,  5.9780,  5.7091,  2.8011, -0.1083,
          -0.1083, -0.1083,  2.7997,  2.8011,  5.9794, -3.0163,  3.0700,
          -3.0163, -0.1083, -5.9257, -5.9257,  3.0723,  0.1643, -3.0177,
          -5.9257, -0.1083, -0.1083,  3.0723,  3.0723, -3.0177, -3.0177,
          -0.1083, -3.0163, -5.9257, -5.9257, -5.9257, -3.0177, -0.1083,
          -3.0163, -5.9257, -5.9257,  0.1643,  3.07

In [80]:
submodel(t.tensor([[15, 27]]))

tensor([[[-3.0177, -5.9257,  3.0724,  0.1643, -5.9257, -5.9257, -0.1083,
          -3.0163,  3.0724,  0.1643, -5.9257, -5.9257, -0.1083, -3.0163,
          -3.0177, -5.9257, -5.9257, -5.9257,  3.0700,  3.0700,  0.1606,
           0.1606,  6.2507,  6.2507,  2.7997, -0.1083,  8.8897,  5.9817,
          -0.1083, -0.1083, -0.1083, -3.0163,  5.9817,  5.9817, -3.0163,
          -0.1083, -0.1083, -3.0163, -0.1083, -0.1083, -3.0163, -0.1083,
           3.0700,  3.0700,  3.0700,  5.9780,  9.1600, 12.0680,  2.7997,
          -0.1083,  5.9804,  3.0723, -3.0177, -3.0177, -3.0177, -5.9257,
           3.0724,  3.0723, -5.9257, -3.0177, -3.0177, -5.9257, -0.1083,
          -0.1083, -5.9257, -3.0177,  0.1606,  0.1606,  3.0700,  5.9780,
           6.2507,  9.1587,  5.7091,  2.8011,  2.7997, -0.1083, -0.1083,
          -0.1083, -0.1083, -3.0163, -0.1083, -0.1083, -3.0163, -0.1083,
          -0.1083, -3.0163,  2.8011,  2.8011, -3.0163, -0.1083,  3.0700,
           3.0700,  5.9794,  8.8874,  3.0700,  5.97

In [10]:
test_inputs = t.tensor(list(product(range(data.N), repeat=2)), device=device)
models[0](test_inputs).shape

torch.Size([14400, 1, 120])

In [13]:
data.groups[0].mult_idx(0, 0)

0

In [14]:
test_inputs[:10]

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [0, 5],
        [0, 6],
        [0, 7],
        [0, 8],
        [0, 9]])